# Part 2: Mining the web: Web APIs

We hope the preceding exercise was painful: even with tools to process HTML, it is rough downloading raw HTML and trying to extract information from it!

> Can you think of any other reasons why scraping websites for data in this way is not a good idea?

Luckily, many websites provide an application programming interface (API) for querying their data or otherwise accessing their services from your programs. For instance, Twitter provides a web API for gathering tweets, Flickr provides one for gathering image data, and Github for accessing information about repository histories.

These kinds of web APIs are much easier to use than, for instance, the preceding technique which scrapes raw web pages and then has to parse the resulting HTML. Moreover, there are more scalable in the sense that the web servers can transmit structured data in a less verbose form than raw HTML.

As a starting example, here is some code to look at the activity on Github related to the public version of our course's materials.

In [1]:
import requests

response = requests.get ('https://api.github.com/repos/cse6040/labs-fa17/events')

print ("==> .headers:", response.headers, "\n")

==> .headers: {'Date': 'Tue, 24 Oct 2017 03:48:26 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '200 OK', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '58', 'X-RateLimit-Reset': '1508820012', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding', 'ETag': 'W/"d53c46c2dfc971561af4fe0268891c8f"', 'Last-Modified': 'Fri, 20 Oct 2017 04:47:45 GMT', 'X-Poll-Interval': '60', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval', 'Access-Control-Allow-Origin': '*', 'Expect-CT': 'max-age=2592000, report-uri="https://api.github.com/_private/browser/errors"', 'Content-Security-Policy': "default-src 'none'", 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Content-Type-Options': 'nos

Note the `Content-Type` of the response:

In [2]:
print (response.headers['Content-Type'])

application/json; charset=utf-8


The response is in JSON format, which is an open format for exchanging semi-structured data. (JSON stands for **J**ava**S**cript **O**bject **N**otation.) JSON is designed to be human-readable and machine-readable, and maps especially well in Python to nested dictionaries. Let's take a look.

> See also [this tutorial](http://www.w3schools.com/json/) for a JSON primer. JSON is among _the_ universal formats for sharing data on the web; see, for instance, https://www.sitepoint.com/10-example-json-files/.

In [4]:
import json
print(type(response.json ()))
print(json.dumps(response.json(), sort_keys=True, indent=2))

<class 'list'>
[
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/5316640?",
      "display_login": "rvuduc",
      "gravatar_id": "",
      "id": 5316640,
      "login": "rvuduc",
      "url": "https://api.github.com/users/rvuduc"
    },
    "created_at": "2017-10-20T04:47:45Z",
    "id": "6741980967",
    "org": {
      "avatar_url": "https://avatars.githubusercontent.com/u/31073927?",
      "gravatar_id": "",
      "id": 31073927,
      "login": "cse6040",
      "url": "https://api.github.com/orgs/cse6040"
    },
    "payload": {
      "before": "3aa6cf03bb6b44fc4a6ccd6aacef953365744042",
      "commits": [
        {
          "author": {
            "email": "richie@cc.gatech.edu",
            "name": "Richard (Rich) Vuduc"
          },
          "distinct": true,
          "message": "Added Lab 9",
          "sha": "32b31bd35f906ef6d2121e961ce466b652d4d671",
          "url": "https://api.github.com/repos/cse6040/labs-fa17/commits/32b31bd35f906ef6d212

**Exercise 0.** It should be self-evident that the JSON response above consists of a sequence of records, which we will refer to as _events_. Each event is associated with an _actor_. Write some code to extract a dictionary of all actors, where the key is the actor's login and the value is the actor's URL.

In [11]:
def extract_actors (json_github_events):
    """Given JSON records for events in a GitHub repo,
    returns a dictionary of the actors and their URLs.
    """
# YOUR CODE HERE
    actors_dict = {}
    for i in range(len(json_github_events)):
        key_login = json_github_events[i]['actor']['login']
        value_URL = json_github_events[i]['actor']['url']
        actors_dict[key_login] = value_URL

    return actors_dict


In [12]:
actor_urls = extract_actors(response.json ())

for actor, url in actor_urls.items ():
    print ('{}: {}'.format(actor, url))
    assert url == "https://api.github.com/users/{}".format(actor)

rvuduc: https://api.github.com/users/rvuduc
dthai91: https://api.github.com/users/dthai91
Augus-Kong: https://api.github.com/users/Augus-Kong


**Exercise 1.** Write some code that goes to each actor's URL and determines their name. If an actor URL is invalid, that actor should not appear in the output.

In [19]:
def lookup_names (actor_urls):
    """Given a dictionary of (actor, url) pairs, looks up the JSON at
    the URL and extracts the user's name (if any). Returns a new
    dictionary of (actor, name) pairs.
    """
# YOUR CODE HERE
    actor_name_dict = {}
    for actor, URL in actor_urls.items():
        response = requests.get(URL).json()
#         if response['message'] == 'Not Found':
#             continue

        name = response['name']
        actor_name_dict[actor] = name
        
    return actor_name_dict


In [20]:
actor_names = lookup_names (actor_urls)

for actor, name in actor_names.items ():
    print ("{}: {}".format (actor, name))
    
assert actor_names['rvuduc'] == 'Rich Vuduc (personal account)'

rvuduc: Rich Vuduc (personal account)
dthai91: Duc Thai
Augus-Kong: XIANGYU KONG


That's the end of this notebook. Processing JSON is fairly straightforward, because it maps very naturally to nested dictionaries in Python. You might search the web for other sources of JSON data, including [this one](https://www.yelp.com/dataset/challenge), and do your own processing!